<a href="https://colab.research.google.com/github/ds-joy/ML-Practice/blob/master/potato_leaf_diseases/potato_diseases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
# from google.colab import drive
# drive.mount('/content/drive')

In [28]:
# ! pip install -q kaggle

In [29]:
# from google.colab import files
# files.upload()

In [30]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [31]:
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets list

ref                                                          title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
-----------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
victorsoeiro/netflix-tv-shows-and-movies                     Netflix TV Shows and Movies                           2MB  2022-05-15 00:01:23           8497        265  1.0              
devansodariya/student-performance-data                       Student Performance Dataset                           7KB  2022-05-26 13:55:09           4195        145  0.9705882        
iamsouravbanerjee/software-professional-salaries-2022        Salary Dataset - 2022                               526KB  2022-06-15 17:13:05           1538         40  1.0              
paradisejoy/top-hits-spotify-from-20002019                   Top Hits Spoti

In [32]:
# ! kaggle datasets download -d arjuntejaswi/plant-village

In [33]:
# !mkdir potato

In [34]:
# !unzip plant-village.zip -d potato

In [35]:
# !rm -rf /content/potato/PlantVillage/Tomato__Tomato_mosaic_virus/
# !rm -rf /content/potato/PlantVillage/Pepper__bell___Bacterial_spot/
# !rm -rf /content/potato/PlantVillage/Pepper__bell___healthy/


In [36]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt

In [37]:
IMAGE_SIZE = 256
BATCH_SIZE = 32

In [38]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/potato/PlantVillage",
    shuffle=True,
    image_size = (IMAGE_SIZE, IMAGE_SIZE),
    batch_size = BATCH_SIZE
)

Found 2151 files belonging to 3 classes.


In [39]:
class_names = dataset.class_names
class_names

['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy']

In [40]:
len(dataset)

68